In [10]:
import numpy as np
import os
import re
import random
import json
import sys
import time
from datetime import datetime
import pandas as pd
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')
from tensorflow.keras.models import Model, load_model, clone_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Dense, Multiply, Add, Lambda, Concatenate, Reshape, Flatten
from tensorflow.keras.initializers import GlorotUniform, RandomUniform, Constant
from tensorflow.keras.callbacks import LambdaCallback
import jax
import jax.numpy as jnp
from jax import grad, jit

In [11]:
epsilon = 0.1
folder = 'D:/OneDrive - Universidad Complutense de Madrid (UCM)/Doctorado/Curriculum_Learning/Multidigit_Addition_Decimal/'

In [12]:
# Verificar si hay GPUs disponibles
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("TensorFlow está utilizando la GPU")
else:
    print("TensorFlow no está utilizando la GPU")

TensorFlow no está utilizando la GPU


In [13]:
# Cargar los módulos preentrenados (unit_module y carry_module)
unit_addition_model = load_model('unit_addition_module.keras')
unit_carry_model = load_model('unit_carry_module.keras')
dec_addition_model = load_model('dec_addition_module.keras')
dec_carry_model = load_model('dec_carry_module.keras')

unit_addition_model.trainable = False
unit_carry_model.trainable = False
dec_addition_model.trainable = False
dec_carry_model.trainable = False

unit_addition_model.name = 'unit_addition_model'
unit_carry_model.name = 'unit_carry_model'
dec_addition_model.name = 'dec_addition_model'
dec_carry_model.name = 'dec_carry_model'

In [14]:
# Cargar las parejas desde el archivo
with open(f"{folder}train_couples.txt", "r") as file:
    train_couples = eval(file.read())

with open(f"{folder}test_dataset.txt", "r") as file:
    test_dataset = eval(file.read())

def generate_test_dataset():
    x_data = []
    y_data = []
    
    for a, b in test_dataset:
        a_dec = a // 10  # Decena del primer número
        a_unit = a % 10  # Unidad del primer número
        b_dec = b // 10  # Decena del segundo número
        b_unit = b % 10  # Unidad del segundo número

        x_data.append([a_dec, a_unit, b_dec, b_unit])  # Entrada

        sum_units = (a_unit + b_unit) % 10
        carry_units = 1 if (a_unit + b_unit) >= 10 else 0
        sum_dec = (a_dec + b_dec + carry_units) % 10
        carry_dec = 1 if (a_dec + b_dec + carry_units) >= 10 else 0
        y_data.append([carry_dec, sum_dec, sum_units])  # Salida
    
    return jnp.array(x_data), jnp.array(y_data)


# Función para leer los datos desde un archivo .txt y generar el dataset de entrenamiento
def generate_train_dataset():
    x_data = []
    y_data = []
    
    for a, b in train_couples:
        a_dec = a // 10  # Decena del primer número
        a_unit = a % 10  # Unidad del primer número
        b_dec = b // 10  # Decena del segundo número
        b_unit = b % 10  # Unidad del segundo número

        x_data.append([a_dec, a_unit, b_dec, b_unit])  # Entrada

        sum_units = (a_unit + b_unit) % 10
        carry_units = 1 if (a_unit + b_unit) >= 10 else 0
        sum_dec = (a_dec + b_dec + carry_units) % 10
        carry_dec = 1 if (a_dec + b_dec + carry_units) >= 10 else 0
        y_data.append([carry_dec, sum_dec, sum_units])  # Salida
    
    return jnp.array(x_data), jnp.array(y_data)

# Función para generar los datos
def generate_final_data():
    x_data = []
    y_data = []
    for a_dec in range(10):
        for a_unit in range(10):
            for b_dec in range(10):
                for b_unit in range(10):
                    x_data.append([a_dec, a_unit, b_dec, b_unit])  # Entrada
                    sum_units = (a_unit + b_unit) % 10
                    carry_units = 1 if (a_unit + b_unit) >= 10 else 0
                    sum_dec = (a_dec + b_dec + carry_units) % 10
                    carry_dec = 1 if (a_dec + b_dec + carry_units) >= 10 else 0
                    y_data.append([carry_dec, sum_dec, sum_units])  # Salida
    return jnp.array(x_data), jnp.array(y_data)

In [15]:
# Función para crear parámetros entrenables (v_0, ..., v_11)
def init_params(epsilon = 0.1):
    v_values_init = jnp.array([1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1], dtype=jnp.float32)
    v_params = {
        f'v{i}': random.uniform(-10, 10) * epsilon + v_values_init[i] for i in range(12)
    }
    return v_params

# Función de pérdida
def loss_fn(params, x, y):
    y_pred_1, y_pred_2, y_pred_3 = model(params, x)
    return jnp.mean((y_pred_1 - y[0,0]) ** 2) + jnp.mean((y_pred_2 - y[0,1]) ** 2) + jnp.mean((y_pred_3 - y[0,2]) ** 2)
    
# Función para entrenar el modelo
def update_params(params, x, y, lr):
    # Asegúrate de usar JAX para los gradientes y operaciones
    gradients = grad(loss_fn)(params, x, y)
    step_loss = loss_fn(params, x, y)
    new_params = jax.tree.map(lambda p, g: p - lr * g, params, gradients)
    return new_params, step_loss

def train_model(params, x_train, y_train, lr=0.01, epochs=100, batch_size=1):
    final_loss = 0
    total_examples = x_train.shape[0]
    
    # Convertir x_train y y_train a arrays de JAX (si aún no lo son)
    x_train = jnp.array(x_train)
    y_train = jnp.array(y_train)
    
    # Entrenar el modelo
    for epoch in range(epochs):  # Número de épocas
        x_batch = x_train[epoch:epoch + 1]
        y_batch = y_train[epoch:epoch + 1]
        params, step_loss = update_params(params, x_batch, y_batch, lr)
        final_loss += step_loss

        if epoch % 10 == 0:
            pred_count, pred_count_train = correct_predictions(params)  # Contamos las predicciones correctas
            print(f"Epoch {epoch}, Loss: {step_loss}, Correct predictions: {pred_count}, Correct predictions train: {pred_count_train}")
            if pred_count_train == 8000:
                break
        
    final_loss = final_loss / epochs
    return params, final_loss

def correct_predictions(params):
    x_test, y_test = generate_test_dataset()
    pred_count = 0
    pred_count_train = 0
    total_examples = x_test.shape[0]
    pred_hundreds, pred_tens, pred_units = model(params, x_test)        
    for i in range(total_examples):
        normalized_pred = [int(jnp.round(pred_hundreds[i].item())),
                           int(jnp.round(pred_tens[i].item())),
                           int(jnp.round(pred_units[i].item()))]
        
        # Obtener los valores a y b de x_test
        a = int(str(x_test[i, 0]) + str(x_test[i, 1]))
        b = int(str(x_test[i, 2]) + str(x_test[i, 3]))
        # Comparar las predicciones con las etiquetas y contar los aciertos
        if normalized_pred[0] == y_test[i, 0] and normalized_pred[1] == y_test[i, 1] and normalized_pred[2] == y_test[i, 2]:
            pred_count += 1
            if (a, b) in train_couples:
                pred_count_train += 1

    return pred_count, pred_count_train

In [16]:
# Modelo dinámico en JAX
def model(params, x):
    units_input = jnp.array(x[:, [1, 3]])  # Columnas 1 y 3 representando unidades y decenas
    units_input = units_input[:, None, :]  # Añade una dimensión extra para la secuencia (N, 1, 2)
                            
    unit_output = jnp.array(unit_addition_model(units_input))  # Asegúrate de que la entrada sea un batch
    unit_carry_output = jnp.array(unit_carry_model(units_input))  # Salida de acarreo de unidades

    # Tomar el valor máximo de las predicciones (argmax en JAX)
    unit_val = jnp.argmax(unit_output, axis=-1)
    carry_unit_val = jnp.argmax(unit_carry_output, axis=-1)

    decs_input = jnp.array(x[:, [0, 2]])
    decs_input = jnp.concatenate([decs_input, carry_unit_val[:, None]], axis=-1)
    decs_input = decs_input[:, None, :]  # Añadir dimensión para la secuencia (N, 1, 3)
    
    dec_output = jnp.array(dec_addition_model(decs_input))  # Salida para decenas
    dec_carry_output = jnp.array(dec_carry_model(decs_input))  # Salida de acarreo de decenas
    
    dec_val = jnp.argmax(dec_output, axis=-1)
    carry_dec_val = jnp.argmax(dec_carry_output, axis=-1)

    # Calcular las salidas combinadas con los parámetros v
    salida_1 = (params['v0'] * carry_dec_val) + (params['v1'] * dec_val) + (params['v2'] * carry_unit_val) + (params['v3'] * unit_val)
    salida_2 = (params['v4'] * carry_dec_val) + (params['v5'] * dec_val) + (params['v6'] * carry_unit_val) + (params['v7'] * unit_val)
    salida_3 = (params['v8'] * carry_dec_val) + (params['v9'] * dec_val) + (params['v10'] * carry_unit_val) + (params['v11'] * unit_val)

    return salida_1, salida_2, salida_3

In [17]:
# Función para leer parámetros de un archivo JSON
def load_params_from_file(filename):
    with open(filename, 'r') as f:
        return json.load(f)

# Guardar el modelo entrenado
def save_trained_model(params, filename, model_dir):
    os.makedirs(model_dir, exist_ok=True)
    file_path = os.path.join(save_model_dir, filename)
    serializable_params = {key: value.tolist() for key, value in params.items()}
    
    with open(file_path, 'w') as f:
        json.dump(serializable_params, f)

class Tee(object):
    def __init__(self, file, mode='w'):
        self.file = open(file, mode)
        self.console = sys.stdout  

    def write(self, data):
        self.console.write(data)   
        self.file.write(data)    

    def flush(self):
        self.console.flush()
        self.file.flush()

    def close(self):
        self.file.close()

In [18]:
x_train, y_train = generate_train_dataset()

save_dir = f"{folder}Results_models/AP_{epsilon}"
save_model_dir = f"{folder}Trained_models/AP_{epsilon}"
folder_path = f'{folder}Parameters/AP_{epsilon}'
date_pattern = r'trainable_model_(\d{4}_\d{2}_\d{2}_\d{2}_\d{2}_\d{2}).json'
files = sorted(
    (f for f in os.listdir(folder_path) if not f.startswith('.')),  # Filtrar archivos ocultos
    key=lambda x: re.search(date_pattern, x).group(1) if re.search(date_pattern, x) else ''
)

for filename in files:
    match = re.search(date_pattern, filename)
    if match:
        current_time = match.group(1)
    else:
        print('Error')
        break
    
    file_path = f"{folder_path}/trainable_model_{current_time}.json"
    with open(file_path, 'rb') as file:
        trainable_model = json.load(file)

    trainable_model_jnp = {key: jnp.array(value) for key, value in trainable_model.items()}
    print(f'Loaded trainable_model_{current_time}.json')
    
    os.makedirs(save_dir, exist_ok=True) 
    results_file = os.path.join(save_dir, f"Results_{current_time}.txt") 
    tee = Tee(results_file, 'w') 
    sys.stdout = tee
    
    try: 
        new_params, average_loss = train_model(trainable_model_jnp, x_train, y_train, lr=0.01, epochs=1000)
        pred_count, pred_count_train = correct_predictions(trainable_model_jnp)

        trained_model_filename = f"trained_model_{current_time}.json"
        save_trained_model(new_params, trained_model_filename, save_model_dir)
        print(f'Saved trained_model_{current_time}.json')

    finally:
        sys.stdout = tee.console
        tee.close()

Loaded trainable_model_2024_12_17_12_18_39.json
Epoch 0, Loss: 26.312023162841797, Correct predictions: 17, Correct predictions train: 14
Epoch 10, Loss: 3.200620651245117, Correct predictions: 243, Correct predictions train: 196
Epoch 20, Loss: 0.0905708521604538, Correct predictions: 4088, Correct predictions train: 3238
Epoch 30, Loss: 0.12740306556224823, Correct predictions: 5413, Correct predictions train: 4316
Epoch 40, Loss: 0.06837225705385208, Correct predictions: 6738, Correct predictions train: 5402
Epoch 50, Loss: 0.0409575030207634, Correct predictions: 5804, Correct predictions train: 4632
Epoch 60, Loss: 0.12922856211662292, Correct predictions: 7618, Correct predictions train: 6078
Epoch 70, Loss: 0.371842622756958, Correct predictions: 7514, Correct predictions train: 6001
Epoch 80, Loss: 0.5409613847732544, Correct predictions: 7609, Correct predictions train: 6078
Epoch 90, Loss: 0.1680581271648407, Correct predictions: 7926, Correct predictions train: 6327
Epoch 10